In [ ]:
# Install and load libraries for image processing steps
# pip install --user sentinel2tools-master.zip
# pip install --user landsatxplore-master.zip
from landsatxplore.api import API
from landsatxplore.earthexplorer import EarthExplorer
from sentinel2download.overlap import Sentinel2Overlap
from sentinel2download.downloader import Sentinel2Downloader
import os
import shutil
from datetime import date, datetime, timedelta
import tarfile 
import glob
import numpy as np
import pandas as pd
from simpledbf import Dbf5
import requests
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

In [3]:
# Search and download Landsat satellite images
def downloadlandsat(startdate, enddate):
    # Initialize a new API instance and get an access key
    username = "username"  # change your EarthExplorer username and password
    password = "password"
    api = API(username, password)
    # 22.13,113.81,22.59,114.52
    # https://github.com/yannforget/landsatxplore/blob/master/landsatxplore/api.py
    # Search for Landsat TM scenes
    scenes = api.search(
        dataset='landsat_ot_c2_l1', bbox=(113.81, 22.13, 114.52, 22.59),
        start_date=startdate,  # start_date='2014-01-01',
        end_date=enddate,   # end_date='2015-12-31',
        max_cloud_cover=20, max_results=1000
    )
    # print(f"{len(scenes)} scenes found.")
    # Log out
    api.logout()
    # Downloading scenes
    if len(scenes) > 0:
        ee = EarthExplorer(username, password)
        for s in scenes:
            ee.download(s['entity_id'], output_dir='D:/WaterQuality/datadownload')
        ee.logout()

In [ ]:
# Find Sentinel overlap tiles
aoi_path = "D:/WaterQuality/aoi/aoi_geojson.json"
overlap = Sentinel2Overlap(aoi_path)
tiles = overlap.overlap()
print(f"Overlapped tiles: {tiles}")

In [ ]:
# Search and download Sentinel satellite images from Google Cloud
def downloadsentinel(startdate, enddate):
    CONSTRAINTS = {'CLOUDY_PIXEL_PERCENTAGE': 20.0, }
    loader = Sentinel2Downloader(api_key="D:/WaterQuality/xxx.json")  # change your Google Cloud Sentinel API key
    loaded = loader.download(product_type="L1C", tiles=['49QGE','49QGF','49QHE','49QHF'], 
                            start_date=startdate, # "2016-01-01"
                            end_date=enddate, # "2016-01-05"
                            output_dir="D:/WaterQuality/datadownload",
                            cores=2, constraints=CONSTRAINTS, full_download=True)

In [ ]:
# Download and extract acolite_py_win_20221114.0.tar.gz
# Save the following text as a txt file "acolite/setting_landsat.txt"
## ACOLITE settings
limit=22.13,113.81,22.59,114.52
inputfile=D:/WaterQuality/datadownload/extract
output=D:/WaterQuality/datadownload/atmocor
dsf_interface_reflectance=False
dsf_residual_glint_correction=True
glint_mask_rhos_threshold=0.15
l2w_parameters=None
l2r_export_geotiff=True

In [ ]:
# Function to preprocess a single Landsat image
def preprocessLandsat(tar):
    # extract tar
    datadir = 'D:/WaterQuality/datadownload'
    os.chdir(datadir)
    file = tarfile.open(tar)
    file.extractall('extract')
    file.close()
    # run acolite
    acolitepath = "D:/WaterQuality/acolite/acolite_py_win/dist/acolite/acolite.exe"
    settingpath = "D:/WaterQuality/acolite/setting_landsat.txt"
    os.system(acolitepath+" --cli --settings="+settingpath)
    def merge_and_mask():
        # merge 7 bands
        os.chdir('atmocor')
        tiflist = glob.glob('*L2R_rhos_*.tif')
        bandorder = [2, 3, 4, 7, 8, 0, 1]
        tiflist = [tiflist[i] for i in bandorder]
        env.workspace = 'D:/WaterQuality/datadownload/atmocor'
        arcpy.CompositeBands_management(tiflist, "compbands.tif")
        # mask land and cloud
        ras = Raster("compbands.tif")
        qaband = Raster(glob.glob(datadir+'/extract/*QA_PIXEL.TIF')[0])
        qaband_m = SetNull(qaband>22200,1)
        qaband_m = FocalStatistics(qaband_m, NbrCircle(3,"CELL"), "MEAN", "NODATA") # expand radius 3
        ras_m = ExtractByMask(ras, qaband_m)
        swir = Raster("compbands.tif\Band_6")
        green = Raster("compbands.tif\Band_3")
        nir = Raster("compbands.tif\Band_5")
        red = Raster("compbands.tif\Band_4")
        ndvi1 = arcpy.sa.Float((red-nir)/(red+nir))
        ndvi1_m = SetNull(ndvi1<0,1)
        ndwi = arcpy.sa.Float((green-swir)/(green+swir))
        ndwi_m = SetNull(ndwi<0,1)
        swir_m = SetNull(swir>0.15,1)
        ras_m = ExtractByMask(ras_m, ndvi1_m)
        ras_m = ExtractByMask(ras_m, ndwi_m)
        ras_m = ExtractByMask(ras_m, swir_m)
        # reproject
        aoi = "D:/WaterQuality/aoi/aoi.shp"
        outfilename = "D:/WaterQuality/reflectance/"+tar.replace(".tar",".tif")
        arcpy.management.ProjectRaster(ras_m, "compbands_p.tif", aoi)            
        arcpy.management.Clip("compbands_p.tif", aoi, "compbands_p_c.tif",                                
                            "#", "#", "NONE","MAINTAIN_EXTENT")
        arcpy.management.Resample("compbands_p_c.tif", outfilename, 0.00028571429)
    merge_and_mask()
    # empty extract and atmocor
    def emptyfolder(folder):
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)    
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
    emptyfolder("D:/WaterQuality/datadownload/extract")
    emptyfolder("D:/WaterQuality/datadownload/atmocor")
    # delete tarfile
    os.chdir(datadir)
    os.unlink(tar)


In [ ]:
# Function to preprocess all Landsat images
def preprocessLandsat_all():
    datadir = 'D:/WaterQuality/datadownload'
    os.chdir(datadir)
    tarlist = glob.glob('*.tar')
    if len(tarlist)>0:
        for tar in tarlist:
            preprocessLandsat(tar)

In [ ]:
# Save the following text as a txt file "acolite/setting_sentinel.txt"
## ACOLITE settings
limit=22.13,113.81,22.59,114.52
inputfile=
output=D:/WaterQuality/datadownload/atmocor
s2_target_res=20
dsf_interface_reflectance=False
dsf_residual_glint_correction=True
glint_mask_rhos_threshold=0.15
l2w_parameters=None
l2r_export_geotiff=True

# Save the following text as a txt file "acolite/setting_sentinel2.txt"
## ACOLITE settings
limit=22.13,113.81,22.59,114.52
inputfile=D:/WaterQuality/datadownload\S2A_MSIL1C_20240717T025551_N0510_R032_T49QHE_20240717T053551.SAFE
output=D:/WaterQuality_LandsatSentinel/datadownload/atmocor
s2_target_res=20
dsf_interface_reflectance=False
dsf_residual_glint_correction=True
glint_mask_rhos_threshold=0.15
l2w_parameters=None
l2r_export_geotiff=True

In [ ]:
# Function to preprocess a single Sentinel image
def preprocessSentinel(safefolder):
    datadir = 'D:/WaterQuality/datadownload'
    os.chdir(datadir)
    # run acolite
    settingtemp = "D:/WaterQuality/acolite/setting_sentinel.txt"
    settingpath = "D:/WaterQuality/acolite/setting_sentinel2.txt"
    # Read in the file
    with open(settingtemp, 'r') as file:
        filedata = file.read()
        filedata = filedata.replace('inputfile=', 'inputfile='+os.path.join(datadir,safefolder))
    # Write the file out again
    with open(settingpath, 'w') as file:
        file.write(filedata)
    acolitepath = "D:/WaterQuality/acolite/acolite_py_win/dist/acolite/acolite.exe"
    os.system(acolitepath+" --cli --settings="+settingpath)
    def merge_and_mask():
        # merge 7 bands
        os.chdir('atmocor')
        tiflist = glob.glob('*L2R_rhos_*.tif')
        if len(tiflist)==0: # if acolite does not produce any files
            return
        bandorder = [2, 3, 4, 5, 10, 0, 1]
        tiflist = [tiflist[i] for i in bandorder]
        env.workspace = 'D:/WaterQuality/datadownload/atmocor'
        arcpy.CompositeBands_management(tiflist, "compbands.tif")
        arcpy.management.Resample("compbands.tif", "compbands_r.tif", 30)
        # mask land and cloud
        ras = Raster("compbands_r.tif")
        swir = Raster("compbands_r.tif\Band_6")
        green = Raster("compbands_r.tif\Band_3")
        nir = Raster("compbands_r.tif\Band_5")
        red = Raster("compbands_r.tif\Band_4")
        cloud_m = SetNull((red>0.2)&(nir>0.2),1)
        cloud_m = FocalStatistics(cloud_m, NbrCircle(3,"CELL"), "MEAN", "NODATA") # expand radius 3
        ndvi1 = arcpy.sa.Float((red-nir)/(red+nir))
        ndvi1_m = SetNull(ndvi1<0,1)
        ndwi2 = arcpy.sa.Float((green-swir)/(green+swir))
        ndwi2_m = SetNull(ndwi2<0,1)
        swir_m = SetNull(swir>0.15,1)
        nir_m = SetNull((nir>0.03)&(red>0.08)&(ndwi2_m==1)&(swir_m==1)&(cloud_m==1),1) # remaining haze
        nir_m = FocalStatistics(nir_m, NbrCircle(1,"CELL"), "MEAN", "NODATA") # expand radius 1
        ras_m = ExtractByMask(ras, cloud_m)
        ras_m = ExtractByMask(ras_m, ndvi1_m)
        ras_m = ExtractByMask(ras_m, ndwi2_m)
        ras_m = ExtractByMask(ras_m, swir_m)
        ras_m = ExtractByMask(ras_m, nir_m)
        # reproject
        aoi = "D:/WaterQuality/aoi/aoi.shp"
        outfilename = "D:/WaterQuality/reflectance/"+safefolder.replace(".SAFE",".tif")
        arcpy.management.ProjectRaster(ras_m, "compbands_p.tif", aoi)
        arcpy.management.Clip("compbands_p.tif", aoi, "compbands_p_c.tif",                                
                            "#", "#", "NONE","MAINTAIN_EXTENT")
        arcpy.management.Resample("compbands_p_c.tif", outfilename, 0.00028571429)
    merge_and_mask()
    # empty extract and atmocor
    def emptyfolder(folder):
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)    
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
    emptyfolder("D:/WaterQuality/datadownload/atmocor")
    # delete whole safefolder
    os.chdir(datadir)
    shutil.rmtree(safefolder)

In [ ]:
# Function to preprocess all Sentinel images
def preprocessSentinel_all():
    datadir = 'D:/WaterQuality/datadownload'
    os.chdir(datadir)
    safelist = glob.glob('*.safe')
    if len(safelist)>0:
        for safefolder in safelist:
            preprocessSentinel(safefolder)

In [ ]:
# Function to get dates in each month
from datetime import date, datetime, timedelta
def monthstart(year, month):
    first_date = datetime(year, month, 1)
    return first_date.strftime("%Y-%m-%d")
def monthmid1(year, month):
    mid_date = datetime(year, month, 15)
    return mid_date.strftime("%Y-%m-%d")
def monthmid2(year, month):
    mid_date = datetime(year, month, 16)
    return mid_date.strftime("%Y-%m-%d")
def monthend(year, month):
    if month == 12:
        last_date = datetime(year, month, 31)
    else:
        last_date = datetime(year, month + 1, 1) + timedelta(days=-1)
    return last_date.strftime("%Y-%m-%d")

In [ ]:
# Run the functions to download and preprocess all Landsat and Sentinel imagery
for year in [2020,2021,2022]:
    for month in range(1,13):
        downloadlandsat(monthstart(year, month), monthend(year, month))
        preprocessLandsat_all()
        downloadsentinel(monthstart(year, month), monthend(year, month))
        preprocessSentinel_all()

In [ ]:
# Remove Tier 2 Landsat imagery
def removeLandsatT2():
    os.chdir("D:/WaterQuality/reflectance")
    Tier2list = glob.glob('LC*T2.*')
    if len(Tier2list)>0:
        for T2file in Tier2list:
            os.unlink(T2file)
# removeLandsatT2()

In [ ]:
# Rename all Landsat imagery
def renameLandsat_all():
    os.chdir("D:/WaterQuality/reflectance")
    Landsatlist = glob.glob('LC*')
    for Landsatfile in Landsatlist:
        nfilename = Landsatfile[0:25]+Landsatfile[40:] # first 25 characters & from 40 to end
        os.rename(Landsatfile, nfilename)
# renameLandsat_all()

In [ ]:
# Rename all Sentinel imagery
def renameSentinel_all():
    os.chdir("D:/WaterQuality/reflectance")
    Sentinellist = glob.glob('S2*')
    for Sentinelfile in Sentinellist:
        nfilename = Sentinelfile[0:19]+Sentinelfile[37:44]+Sentinelfile[60:]
        if os.path.isfile(nfilename) == True:
            nfilename = Sentinelfile[0:19]+Sentinelfile[37:44]+'a'+Sentinelfile[60:]
        os.rename(Sentinelfile, nfilename)
# renameSentinel_all()

In [ ]:
# Mosaic tiles acquired on the same day
def mosaictiles(): 
    os.chdir("D:/WaterQuality/reflectance")
    env.workspace = "D:/WaterQuality/reflectance"
    Landsatlist = glob.glob('LC*')
    Landsatdatelist = [i[17:25] for i in Landsatlist]
    Sentinellist = glob.glob('S2*')
    Sentineldatelist = [i[11:19] for i in Sentinellist]
    datelist = sorted(list(set(Landsatdatelist+Sentineldatelist))) # get unique date
    imglist = glob.glob('*.tif')
    for d in datelist:
        img_match = [img for img in imglist if d in img]
        outfolder = "D:/WaterQuality/preprocess_finish"
        outfilename = "LandsatSentinel_"+d+".tif"
        if len(img_match)==1:
            arcpy.management.CopyRaster(img_match[0], os.path.join(outfolder, outfilename))
        if len(img_match)>1:
            arcpy.MosaicToNewRaster_management(img_match,outfolder,outfilename,"","32_BIT_FLOAT","","7","MEAN","")
# mosaictiles()

In [ ]:
# Remove images that cover too few valid pixels
def deleteimage_lowvalid():
    os.chdir("D:/WaterQuality/preprocess_finish")
    env.workspace = "D:/WaterQuality/preprocess_finish"
    imglist = glob.glob('*.tif')
    for img in imglist:
        ras_np = arcpy.RasterToNumPyArray(img,"","","",-9999)[0]
        if (ras_np != -9999).sum() < (2390000*0.2): # largest valid count = 2390000
            arcpy.management.Delete(img)
# deleteimage_lowvalid()

In [ ]:
# Extract pixel values based on the monitoring station locations
def extractpixel(imgname):
    os.chdir("D:/WaterQuality/preprocess_finish")
    env.workspace = "D:/WaterQuality/preprocess_finish"
    imgdate = imgname[16:24]
    station_shp = "D:/WaterQuality/stationdata/MonitoringStation_wgs84_76.shp"
    extract_dbf = "D:/WaterQuality/extract/extract.dbf"
    arcpy.sa.Sample(imgname, station_shp, extract_dbf, "BILINEAR", "FID")
    extract_df = Dbf5(extract_dbf).to_dataframe()
    extract_df["monitoring"] = range(0,len(extract_df))
    arcpy.management.Delete(extract_dbf)
    extract_df.columns = ['Monitoring','X','Y','B1','B2','B3','B4','B5','B6','B7','monitoring']
    station_df = Dbf5("D:/WaterQuality/stationdata/MonitoringStation_wgs84_76.dbf").to_dataframe()
    station_df["monitoring"] = range(0,len(station_df))
    station_df = station_df[["monitoring","WaterStati"]]
    extract_df = extract_df.merge(station_df, on="monitoring")
    extract_df = extract_df.drop(columns=['Monitoring','X','Y','monitoring'])
    extract_df = extract_df[extract_df["B1"] > -1] # remove no data (-9999) rows
    extract_df["Date"] = imgdate
    return extract_df

In [ ]:
# Extract and save the pixel values as a csv file
os.chdir("D:/WaterQuality/preprocess_finish")
imglist = glob.glob('*.tif')
dflist = [extractpixel(img) for img in imglist]
extract_df_all = pd.concat(dflist)
extract_df_all.to_csv("D:/WaterQuality/extract/extract_df_all.csv")

In [7]:
# Combine station data different years
os.chdir("D:/WaterQuality/stationdata")
csvlist = glob.glob('marine-historical-*.csv')
csvlist = [pd.read_csv(c) for c in csvlist]
df = pd.concat(csvlist)
df.to_csv("marine-historical-2010-2022.csv")
# Additional manual step: subset only surface water and replace <0.2 to 0.1, <0.5 to 0.25

In [ ]:
# Merge image pixel value and station data that are same day
stationdata_df = pd.read_csv("D:/WaterQuality/stationdata/marine-historical-2010-2022.csv")
extract_df_all = pd.read_csv("D:/WaterQuality/extract/extract_df_all.csv", index_col=0)
stationdata_df = stationdata_df[["Station","Dates","Chlorophyll-a (µg/L)","Suspended Solids (mg/L)","Turbidity (NTU)"]]
stationdata_df.columns = ["WaterStati", "Date", "Chla", "SS", "Tur"]
stationdata_df["Date"] = pd.to_datetime(stationdata_df["Date"], format='%d/%m/%Y').dt.strftime('%Y%m%d').astype(int)
img_stationdata_merge = stationdata_df.merge(extract_df_all, on=["WaterStati","Date"])
img_stationdata_merge.to_csv("D:/WaterQuality/extract/img_stationdata_merge.csv")


In [194]:
# Create independent variables
df = pd.read_csv("D:/WaterQuality/extract/img_stationdata_merge.csv", index_col=0)
bands = ['B' + str(b) for b in [*range(1,8)]]
wl = [443,490,560,660,865,1610,2195]  #wavelength in nm

# Multiply 10
for i in bands:
  df[i] = df[i]*10
# Square and cubic
for i in bands:
  df[i+'_2'] = df[i]**2
  df[i+'_3'] = df[i]**3
# Two-band ratio
for i in bands:
  for j in bands:
    if (i != j) & (i < j):
      df['NR_'+i+j] = ((df[i] - df[j]) / (df[i] + df[j])).clip(lower=-1.0, upper=1.0)
# Three-band ratio
for i in range(0,7):
  for j in range(0,7):
    for k in range(0,7):
      if (j == i+1) & (k == j+1):
        df['TB_'+bands[i]+bands[j]+bands[k]] = (((1/df[bands[i]])-(1/df[bands[j]]))*df[bands[k]]).clip(lower=-1.0, upper=1.0)
# Line height algorithm
for i in range(0,7):
  for j in range(0,7):
    for k in range(0,7):
      if (j == i+1) & (k == j+1):
        df['LH_'+bands[i]+bands[j]+bands[k]] = df[bands[j]] - df[bands[i]] - ((df[bands[k]] - df[bands[i]]) * ((wl[j]-wl[i])/(wl[k]-wl[i])))

df.to_csv('D:/WaterQuality/extract/df_variable.csv')

In [211]:
df = pd.read_csv("D:/WaterQuality/extract/df_variable.csv", index_col=0)
df = df.drop(columns = ["WaterStati","Date"])
df_var = df.iloc[:,3:]

In [ ]:
# Install and load libraries for neural network steps
import tensorflow as tf
from keras import backend as K
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import statsmodels.api as sm

In [3]:
# Function for stepwise variable selection
# Modified from https://datascience.stackexchange.com/questions/24405/how-to-do-stepwise-regression-using-sklearn/24447#24447
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.1, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - pandas.DataFrame with the target column
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    y = y.to_numpy()
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype='float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.index[pvalues.argmax()]
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [4]:
# Function to obtain model performance based on cross validation
wq = ['Chla','SS','Tur']
def model_cv(wq_name):
    y = df[wq_name]
    step_var = stepwise_selection(df_var, y, verbose=False)
    X = df[step_var]
    kfold = KFold(n_splits=10, shuffle=True, random_state=0)
    df_cv = pd.DataFrame()
    for train, test in kfold.split(X, y):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(6, activation='sigmoid'),
            tf.keras.layers.Dense(3, activation='sigmoid'),
            tf.keras.layers.Dense(1)
        ])
        model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam())
        model.fit(X.iloc[train,], y[train], epochs=2000)
        y_predict = model.predict(X.iloc[test,]).flatten()
        y_test = y[test]
        df_cv1 = pd.DataFrame({"y_predict":y_predict, "y_test":y_test})
        df_cv = pd.concat([df_cv,df_cv1])
    corr_test = np.corrcoef(df_cv["y_test"], df_cv["y_predict"])[0, 1]
    rmse_test = mean_squared_error(df_cv["y_test"], df_cv["y_predict"], squared=False)
    mae_test = mean_absolute_error(df_cv["y_test"], df_cv["y_predict"])
    model_cv_df = pd.DataFrame({'WQ': [wq_name], 'var':[step_var], 'corr_test': [corr_test], 
                                'rmse_test': [rmse_test], 'mae_test': [mae_test]})
    return(model_cv_df)

In [ ]:
# Obtain model performance
model_cv_Chla = model_cv('Chla')
model_cv_SS = model_cv('SS')

In [ ]:
# Chla model
wq_name = "Chla"
y = df[wq_name]
step_var = stepwise_selection(df_var, y, verbose=False)
X = df[step_var]
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6, activation='sigmoid'),
    tf.keras.layers.Dense(3, activation='sigmoid'),
    tf.keras.layers.Dense(1)
])
model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam())
model.fit(X, y, epochs=2000)
model.save("D:/WaterQuality/extract/Chla_model.keras")

# SS model
wq_name = "SS"
y = df[wq_name]
step_var = stepwise_selection(df_var, y, verbose=False)
X = df[step_var]
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6, activation='sigmoid'),
    tf.keras.layers.Dense(3, activation='sigmoid'),
    tf.keras.layers.Dense(1)
])
model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam())
model.fit(X, y, epochs=2000)
model.save("D:/WaterQuality/extract/SS_model.keras")

In [315]:
# print model weights from the trained models
Chla_model = tf.keras.models.load_model("D:/WaterQuality/extract/Chla_model.keras")
pd.Series(Chla_model.get_weights()).to_json("D:/WaterQuality/extract/Chla_modelweight.json")
SS_model = tf.keras.models.load_model("D:/WaterQuality/extract/SS_model.keras")
pd.Series(SS_model.get_weights()).to_json("D:/WaterQuality/extract/SS_modelweight.json")
# OR pd.read_json("D:/WaterQuality/extract/Chla_modelweight.json", typ="series")

In [ ]:
# Apply model to predict all imagery
import os
import glob
import numpy as np
import pandas as pd
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

In [7]:
# Function to predict Chla
def predictChla(imgname):
        outfolder = "D:/WaterQuality/predict"
        os.chdir(outfolder)
        env.workspace = outfolder
        # ANN layers: 14, 6, 3, 1
        p = pd.read_json("D:/WaterQuality/extract/Chla_modelweight.json", typ="series")
        # Chla ['NR_B2B4', 'NR_B2B6', 'NR_B3B6', 'NR_B3B4', 'TB_B1B2B3', 
        # 'TB_B4B5B6', 'B2_3', 'B5', 'B4_2', 'B3_3', 'TB_B2B3B4', 'B6_3', 'NR_B1B6', 'B6_2']
        outname = imgname.replace("preprocess_finish", "predict").replace("LandsatSentinel_20", "Chla_20")
        b1 = Raster(imgname+"/Band_1")*10
        b2 = Raster(imgname+"/Band_2")*10
        b3 = Raster(imgname+"/Band_3")*10
        b4 = Raster(imgname+"/Band_4")*10
        b5 = Raster(imgname+"/Band_5")*10
        b6 = Raster(imgname+"/Band_6")*10
        ras_0 = b1*0
        ras_neg1 = ras_0 - 1
        ras_1 = ras_0 + 1
        v1 = CellStatistics([CellStatistics([(b2-b4)/(b2+b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v2 = CellStatistics([CellStatistics([(b2-b6)/(b2+b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v3 = CellStatistics([CellStatistics([(b3-b6)/(b3+b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v4 = CellStatistics([CellStatistics([(b3-b4)/(b3+b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v5 = CellStatistics([CellStatistics([(((1/b1)-(1/b2))*b3),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v6 = CellStatistics([CellStatistics([(((1/b4)-(1/b5))*b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v7 = b2 ** 3
        v8 = b5
        v9 = b4 ** 2
        v10 = b3 ** 3
        v11 = CellStatistics([CellStatistics([(((1/b2)-(1/b3))*b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v12 = b6 ** 3
        v13 = CellStatistics([CellStatistics([(b1-b6)/(b1+b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v14 = b6 ** 2

        h1n1 = (Exp((p[1][0]+v1*p[0][0][0]+v2*p[0][1][0]+v3*p[0][2][0]+v4*p[0][3][0]+v5*p[0][4][0]+
                v6*p[0][5][0]+v7*p[0][6][0]+v8*p[0][7][0]+v9*p[0][8][0]+v10*p[0][9][0]+
                v11*p[0][10][0]+v12*p[0][11][0]+v13*p[0][12][0]+v14*p[0][13][0])*(-1))+1)**-1
        h1n2 = (Exp((p[1][1]+v1*p[0][0][1]+v2*p[0][1][1]+v3*p[0][2][1]+v4*p[0][3][1]+v5*p[0][4][1]+
                v6*p[0][5][1]+v7*p[0][6][1]+v8*p[0][7][1]+v9*p[0][8][1]+v10*p[0][9][1]+
                v11*p[0][10][1]+v12*p[0][11][1]+v13*p[0][12][1]+v14*p[0][13][1])*(-1))+1)**-1
        h1n3 = (Exp((p[1][2]+v1*p[0][0][2]+v2*p[0][1][2]+v3*p[0][2][2]+v4*p[0][3][2]+v5*p[0][4][2]+
                v6*p[0][5][2]+v7*p[0][6][2]+v8*p[0][7][2]+v9*p[0][8][2]+v10*p[0][9][2]+
                v11*p[0][10][2]+v12*p[0][11][2]+v13*p[0][12][2]+v14*p[0][13][2])*(-1))+1)**-1
        h1n4 = (Exp((p[1][3]+v1*p[0][0][3]+v2*p[0][1][3]+v3*p[0][2][3]+v4*p[0][3][3]+v5*p[0][4][3]+
                v6*p[0][5][3]+v7*p[0][6][3]+v8*p[0][7][3]+v9*p[0][8][3]+v10*p[0][9][3]+
                v11*p[0][10][3]+v12*p[0][11][3]+v13*p[0][12][3]+v14*p[0][13][3])*(-1))+1)**-1
        h1n5 = (Exp((p[1][4]+v1*p[0][0][4]+v2*p[0][1][4]+v3*p[0][2][4]+v4*p[0][3][4]+v5*p[0][4][4]+
                v6*p[0][5][4]+v7*p[0][6][4]+v8*p[0][7][4]+v9*p[0][8][4]+v10*p[0][9][4]+
                v11*p[0][10][4]+v12*p[0][11][4]+v13*p[0][12][4]+v14*p[0][13][4])*(-1))+1)**-1
        h1n6 = (Exp((p[1][5]+v1*p[0][0][5]+v2*p[0][1][5]+v3*p[0][2][5]+v4*p[0][3][5]+v5*p[0][4][5]+
                v6*p[0][5][5]+v7*p[0][6][5]+v8*p[0][7][5]+v9*p[0][8][5]+v10*p[0][9][5]+
                v11*p[0][10][5]+v12*p[0][11][5]+v13*p[0][12][5]+v14*p[0][13][5])*(-1))+1)**-1

        h2n1 = (Exp((p[3][0]+h1n1*p[2][0][0]+h1n2*p[2][1][0]+h1n3*p[2][2][0]+
                h1n4*p[2][3][0]+h1n5*p[2][4][0]+h1n6*p[2][5][0])*(-1))+1)**-1
        h2n2 = (Exp((p[3][1]+h1n1*p[2][0][1]+h1n2*p[2][1][1]+h1n3*p[2][2][1]+
                h1n4*p[2][3][1]+h1n5*p[2][4][1]+h1n6*p[2][5][1])*(-1))+1)**-1
        h2n3 = (Exp((p[3][2]+h1n1*p[2][0][2]+h1n2*p[2][1][2]+h1n3*p[2][2][2]+
                h1n4*p[2][3][2]+h1n5*p[2][4][2]+h1n6*p[2][5][2])*(-1))+1)**-1

        pred = CellStatistics([p[5][0]+h2n1*p[4][0][0]+h2n2*p[4][1][0]+h2n3*p[4][2][0],ras_0], "MAXIMUM")
        arcpy.management.CopyRaster(pred, outname)

In [8]:
# Function to predict SS
def predictSS(imgname):
        outfolder = "D:/WaterQuality/predict"
        os.chdir(outfolder)
        env.workspace = outfolder
        # ANN layers: 9, 6, 3, 1
        p = pd.read_json("D:/WaterQuality/extract/SS_modelweight.json", typ="series")
        # SS ['TB_B2B3B4', 'LH_B4B5B6', 'B3_3', 'B4_2', 'LH_B5B6B7', 'TB_B3B4B5', 'NR_B5B6', 'NR_B1B4', 'B2_3']
        outname = imgname.replace("preprocess_finish", "predict").replace("LandsatSentinel_20", "SuSo_20")
        b1 = Raster(imgname+"/Band_1")*10
        b2 = Raster(imgname+"/Band_2")*10
        b3 = Raster(imgname+"/Band_3")*10
        b4 = Raster(imgname+"/Band_4")*10
        b5 = Raster(imgname+"/Band_5")*10
        b6 = Raster(imgname+"/Band_6")*10
        b7 = Raster(imgname+"/Band_7")*10
        ras_0 = b1*0
        ras_neg1 = ras_0 - 1
        ras_1 = ras_0 + 1
        v1 = CellStatistics([CellStatistics([(((1/b2)-(1/b3))*b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v2 = b5-b4-((b6-b4)*((865-660)/(1610-660)))
        v3 = b3 ** 3
        v4 = b4 ** 2
        v5 = b6-b5-((b7-b5)*((1610-865)/(2195-865)))
        v6 = CellStatistics([CellStatistics([(((1/b3)-(1/b4))*b5),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v7 = CellStatistics([CellStatistics([(b5-b6)/(b5+b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v8 = CellStatistics([CellStatistics([(b1-b4)/(b1+b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
        v9 = b2 ** 3

        h1n1 = (Exp((p[1][0]+v1*p[0][0][0]+v2*p[0][1][0]+v3*p[0][2][0]+v4*p[0][3][0]+v5*p[0][4][0]+
                v6*p[0][5][0]+v7*p[0][6][0]+v8*p[0][7][0]+v9*p[0][8][0])*(-1))+1)**-1
        h1n2 = (Exp((p[1][1]+v1*p[0][0][1]+v2*p[0][1][1]+v3*p[0][2][1]+v4*p[0][3][1]+v5*p[0][4][1]+
                v6*p[0][5][1]+v7*p[0][6][1]+v8*p[0][7][1]+v9*p[0][8][1])*(-1))+1)**-1
        h1n3 = (Exp((p[1][2]+v1*p[0][0][2]+v2*p[0][1][2]+v3*p[0][2][2]+v4*p[0][3][2]+v5*p[0][4][2]+
                v6*p[0][5][2]+v7*p[0][6][2]+v8*p[0][7][2]+v9*p[0][8][2])*(-1))+1)**-1
        h1n4 = (Exp((p[1][3]+v1*p[0][0][3]+v2*p[0][1][3]+v3*p[0][2][3]+v4*p[0][3][3]+v5*p[0][4][3]+
                v6*p[0][5][3]+v7*p[0][6][3]+v8*p[0][7][3]+v9*p[0][8][3])*(-1))+1)**-1
        h1n5 = (Exp((p[1][4]+v1*p[0][0][4]+v2*p[0][1][4]+v3*p[0][2][4]+v4*p[0][3][4]+v5*p[0][4][4]+
                v6*p[0][5][4]+v7*p[0][6][4]+v8*p[0][7][4]+v9*p[0][8][4])*(-1))+1)**-1
        h1n6 = (Exp((p[1][5]+v1*p[0][0][5]+v2*p[0][1][5]+v3*p[0][2][5]+v4*p[0][3][5]+v5*p[0][4][5]+
                v6*p[0][5][5]+v7*p[0][6][5]+v8*p[0][7][5]+v9*p[0][8][5])*(-1))+1)**-1

        h2n1 = (Exp((p[3][0]+h1n1*p[2][0][0]+h1n2*p[2][1][0]+h1n3*p[2][2][0]+
                h1n4*p[2][3][0]+h1n5*p[2][4][0]+h1n6*p[2][5][0])*(-1))+1)**-1
        h2n2 = (Exp((p[3][1]+h1n1*p[2][0][1]+h1n2*p[2][1][1]+h1n3*p[2][2][1]+
                h1n4*p[2][3][1]+h1n5*p[2][4][1]+h1n6*p[2][5][1])*(-1))+1)**-1
        h2n3 = (Exp((p[3][2]+h1n1*p[2][0][2]+h1n2*p[2][1][2]+h1n3*p[2][2][2]+
                h1n4*p[2][3][2]+h1n5*p[2][4][2]+h1n6*p[2][5][2])*(-1))+1)**-1

        pred = CellStatistics([p[5][0]+h2n1*p[4][0][0]+h2n2*p[4][1][0]+h2n3*p[4][2][0],ras_0], "MAXIMUM")
        arcpy.management.CopyRaster(pred, outname)

In [7]:
# Apply function to all imagery
imglist = glob.glob('D:/WaterQuality/preprocess_finish/*.tif')
for i in imglist:
    predictChla(i)
    predictSS(i)

In [ ]:
# Create Chla and SS raster for the latest date
aoi_water = "D:/WaterQuality/aoi/aoi_water.shp"
# Chla
chlalist = glob.glob("D:/WaterQuality/predict/Chla_*.tif")
latestimg = chlalist[len(chlalist)-1]
latestimg = latestimg[len(latestimg)-17:len(latestimg)]
mergeras = arcpy.management.MosaicToNewRaster(chlalist, "D:/WaterQuality/predict_display", "merge_"+latestimg, "", "32_BIT_FLOAT", "", 1, "LAST")
mergeras_focal = FocalStatistics(mergeras, NbrRectangle(3,3,"CELL"), "MEDIAN")
outname = ("D:/WaterQuality/predict_display/merge_"+latestimg).replace(".tif", "_smoothclip.tif")
arcpy.management.Clip(mergeras_focal, "", outname, aoi_water, "", "ClippingGeometry")
# SS
sslist = glob.glob("D:/WaterQuality/predict/SuSo_*.tif")
latestimg = sslist[len(sslist)-1]
latestimg = latestimg[len(latestimg)-17:len(latestimg)]
mergeras = arcpy.management.MosaicToNewRaster(sslist, "D:/WaterQuality/predict_display", "merge_"+latestimg, "", "32_BIT_FLOAT", "", 1, "LAST")
mergeras_focal = FocalStatistics(mergeras, NbrRectangle(3,3,"CELL"), "MEDIAN")
outname = ("D:/WaterQuality/predict_display/merge_"+latestimg).replace(".tif", "_smoothclip.tif")
arcpy.management.Clip(mergeras_focal, "", outname, aoi_water, "", "ClippingGeometry")


In [ ]:
# Create datapoints using Fishnet tool in ArcGIS
# Datapoints are used to plot charts in Dashboard
# Extract and append values for the first date
datapoint = "D:/WaterQuality/ArcGISPro/DataPoint.shp"
datapoint_d1 = "D:/WaterQuality/ArcGISPro/DataPoint_d1.shp"
datapoint_d2 = "D:/WaterQuality/ArcGISPro/DataPoint_d2.shp"
datapoint_all = "D:/WaterQuality/ArcGISPro/DataPoint_all.shp"
arcpy.management.Copy(datapoint, datapoint_d1)
arcpy.management.Copy(datapoint, datapoint_d2)
chla_d1 = "D:/WaterQuality/predict/Chla_20130708.tif"
ss_d1 = "D:/WaterQuality/predict/SuSo_20130708.tif"
arcpy.sa.ExtractMultiValuesToPoints(datapoint_d1, chla_d1+" value", "BILINEAR")
arcpy.sa.ExtractMultiValuesToPoints(datapoint_d2, ss_d1+" value", "BILINEAR")
with arcpy.da.UpdateCursor(datapoint_d1, ["value","Date","parameter","latest","DateRange"]) as cursor:
    for row in cursor:
        if row[0] < 0:
            cursor.deleteRow()
        else:
            row[1] = 20130708
            row[2] = "Chla"
            row[3] = 1
            row[4] = "Day"
            cursor.updateRow(row)
with arcpy.da.UpdateCursor(datapoint_d2, ["value","Date","parameter","latest","DateRange"]) as cursor:
    for row in cursor:
        if row[0] < 0:
            cursor.deleteRow()
        else:
            row[1] = 20130708
            row[2] = "SS"
            row[3] = 1
            row[4] = "Day"
            cursor.updateRow(row)
arcpy.management.Copy(datapoint_d1, datapoint_all)
arcpy.management.Append(datapoint_d2, datapoint_all)
with arcpy.da.UpdateCursor(datapoint_d1, ["Date","DateRange"]) as cursor:
    for row in cursor:
        row[0] = round(20130708/100)*100+15
        row[1] = "Month"
        cursor.updateRow(row)
with arcpy.da.UpdateCursor(datapoint_d2, ["Date","DateRange"]) as cursor:
    for row in cursor:
        row[0] = round(20130708/100)*100+15
        row[1] = "Month"
        cursor.updateRow(row)
arcpy.management.Append(datapoint_d1, datapoint_all)
arcpy.management.Append(datapoint_d2, datapoint_all)
arcpy.management.Delete(datapoint_d1)
arcpy.management.Delete(datapoint_d2)

In [ ]:
# Append datapoint from 2nd to latest dates
datapoint = "D:/WaterQuality/ArcGISPro/DataPoint.shp"
datapoint_d1 = "D:/WaterQuality/ArcGISPro/DataPoint_d1.shp"
datapoint_d2 = "D:/WaterQuality/ArcGISPro/DataPoint_d2.shp"
datapoint_all = "D:/WaterQuality/ArcGISPro/DataPoint_all.shp"
chlalist = glob.glob("D:/WaterQuality/predict/Chla_*.tif")
sslist = glob.glob("D:/WaterQuality/predict/SuSo_*.tif")
for i in range(1,len(chlalist)):
    chla_d1 = chlalist[i]
    ss_d1 = sslist[i]
    arcpy.management.Copy(datapoint, datapoint_d1)
    arcpy.management.Copy(datapoint, datapoint_d2)
    arcpy.sa.ExtractMultiValuesToPoints(datapoint_d1, chla_d1+" value", "BILINEAR")
    arcpy.sa.ExtractMultiValuesToPoints(datapoint_d2, ss_d1+" value", "BILINEAR")
    # remove points with no data, else add date and extract list of valid points
    newdatapt = []
    d1 = int(chla_d1[len(chla_d1)-12:len(chla_d1)-4])
    d1_month = round(d1/100)*100+15
    with arcpy.da.UpdateCursor(datapoint_d1, ["value","pt","Date","parameter","latest","DateRange"]) as cursor:
        for row in cursor:
            if row[0] < 0:
                cursor.deleteRow()
            else:
                newdatapt.append(row[1])
                row[2] = d1 # Date
                row[3] = "Chla"
                row[4] = 1 # latest
                row[5] = "Day"
                cursor.updateRow(row)
    # modify values for SS
    with arcpy.da.UpdateCursor(datapoint_d2, ["value","pt","Date","parameter","latest","DateRange"]) as cursor:
        for row in cursor:
            if row[0] < 0:
                cursor.deleteRow()
            else:
                row[2] = d1 # Date
                row[3] = "SS"
                row[4] = 1 # latest
                row[5] = "Day"
                cursor.updateRow(row)    
    # modify latest in datapoint_all
    with arcpy.da.UpdateCursor(datapoint_all, ["pt","latest","Date","DateRange"], "latest = 1") as cursor: # where clause
        for row in cursor:
            if row[0] in newdatapt: # if latest image provides obs on this pt
                row[1] = 0
                if row[3]=="Month" and row[2]==d1_month: # if same month as latest image
                    row[1] = 1
                cursor.updateRow(row)
    arcpy.management.Append(datapoint_d1, datapoint_all)
    arcpy.management.Append(datapoint_d2, datapoint_all)
    # add monthly average data
    with arcpy.da.UpdateCursor(datapoint_d1, ["Date","DateRange"]) as cursor:
        for row in cursor:
            row[0] = d1_month
            row[1] = "Month"
            cursor.updateRow(row)
    with arcpy.da.UpdateCursor(datapoint_d2, ["Date","DateRange"]) as cursor:
        for row in cursor:
            row[0] = d1_month
            row[1] = "Month"
            cursor.updateRow(row)
    arcpy.management.Append(datapoint_d1, datapoint_all)
    arcpy.management.Append(datapoint_d2, datapoint_all)
    arcpy.management.Delete(datapoint_d1)
    arcpy.management.Delete(datapoint_d2)
    print("Finish: "+ str(i)+"/"+str(len(chlalist)))